<a href="https://colab.research.google.com/github/rishikeshr/100-Days-Of-ML-Code/blob/master/async_vector_search_vz_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --upgrade --quiet langchain-elasticsearch langchain-openai tiktoken langchain pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 523.3/523.3 kB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.4/533.4 kB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
from elasticsearch import AsyncElasticsearch, OrjsonSerializer

import nest_asyncio
nest_asyncio.apply()
import numpy as np


import concurrent.futures
import time
import random
import pandas as pd
import json
import os
import aiohttp
from elasticsearch import Elasticsearch
import configparser
import asyncio
import urllib.request
from concurrent.futures import ThreadPoolExecutor, as_completed
import statistics

# embeddings=[]
# for i in range(100):
#     # random_array = np.random.normal(0.0, 1.0, 768)
#     v = np.random.rand(1, 768)
#     vn = v / np.linalg.norm(v, axis=1, keepdims=True)
#     # print(vn)
#     # break
#     embeddings.append(vn.tolist()[0])

# with open("new_embedding_file_100k_random.txt","w") as fp:
#     json.dump(embeddings,fp)

# with open('new_embedding_file_100k_random.txt') as f:
#   vector_lines = json.load(f)

with open('new_embedding_file_100k_random.txt') as f:
   vector_lines = json.load(f)

# Set up Elasticsearch connection
ES_HOSTS = ["https://34.29.60.210:9200"]  # Replace with your Elasticsearch host(s)
ELASTIC_API_KEY = "OGRLVGk1RUI5b2xjak1rbDBMU1I6TDd2enlLUERUMG01dUhfbHJscjFtUQ=="

# Initialize the AsyncElasticsearch client
es = AsyncElasticsearch(
    hosts=ES_HOSTS,
    api_key=ELASTIC_API_KEY,
    verify_certs=False,
    retry_on_timeout=True,
    #http_compress=True,
    connections_per_node=200,
    serializers={"application/json": OrjsonSerializer()}
)

async def run_knn_search(query_vector, index_name, field_name):
    search_body = {
        "_source": False, ##to avoid returning full doc
        "size": 4,
        "terminate_after": 5,
        "knn": {
            "field": field_name,
            "query_vector": query_vector,
            "k": 4,
            "num_candidates": 5
        }
    }

    start_service_time = time.time()  # Start measuring service time
    response = await es.search(index=index_name, body=search_body)
    end_service_time = time.time()  # End measuring service time

    took_time = response['took']  # Time in milliseconds
    service_time = end_service_time - start_service_time  # Service time in seconds

    return took_time, service_time  # Return both took time and service time

async def thread_task(index_name, field_name, vectors_for_thread):
    took_times = []
    service_times = []

    num_queries = len(vectors_for_thread)
    for i in range(num_queries):
        query_vector = vectors_for_thread[i % len(vectors_for_thread)]  # Reuse vectors if needed
        took_time, service_time = await run_knn_search(query_vector, index_name, field_name)
        took_times.append(took_time)
        service_times.append(service_time)

    return took_times, service_times

async def main():
    num_threads = 100  # Number of parallel threads (tasks) ex: 100
    queries_per_thread = 100  # Number of queries each thread will run ex: 1000
    index_name = "tech15m2_random"
    field_name = "vector"

    try:
        # Get the total number of documents in the index
        count_response = await es.count(index=index_name)
        total_docs = count_response['count']

        # Output the total number of documents in the index
        print(f"Total number of documents in the index '{index_name}': {total_docs}")

        # Get index settings to retrieve the number of shards and replicas
        index_settings = await es.indices.get(index=index_name)
        settings = index_settings[index_name]['settings']['index']
        number_of_shards = settings['number_of_shards']
        number_of_replicas = settings['number_of_replicas']

        # Output the number of primary shards and replicas
        print(f"Number of primary shards: {number_of_shards}")
        print(f"Number of replicas: {number_of_replicas}")

        # If total_queries exceeds num_vectors, vectors will be reused
        vectors_per_thread = [vector_lines[i % len(vector_lines)] for i in range(queries_per_thread * num_threads)]

        # Measure total runtime
        start_time = time.time()  # Start time for total runtime

        # Assign vectors to threads
        tasks = []
        for thread_index in range(num_threads):
            start = thread_index * queries_per_thread
            end = start + queries_per_thread
            vectors_for_thread = vectors_per_thread[start:end]
            tasks.append(thread_task(index_name, field_name, vectors_for_thread))

        # Gather the results from all tasks
        all_results = await asyncio.gather(*tasks)

        end_time = time.time()  # End time for total runtime
        total_runtime = end_time - start_time  # Total runtime in seconds

        # Flatten the lists of took_times and service_times, and convert took_times to seconds
        took_times_flat = [time / 1000.0 for sublist in [result[0] for result in all_results] for time in sublist]
        service_times_flat = [time for sublist in [result[1] for result in all_results] for time in sublist]

        min_took_time = min(took_times_flat)
        max_took_time = max(took_times_flat)
        avg_took_time = statistics.mean(took_times_flat)

        min_service_time = min(service_times_flat)
        max_service_time = max(service_times_flat)
        avg_service_time = statistics.mean(service_times_flat)

        # Calculate total number of queries
        total_queries_ran = len(took_times_flat)

        # Calculate average response time
        avg_response_time = total_runtime / total_queries_ran

        # Print the timing statistics in seconds
        print(f"Min took time (seconds): {min_took_time:.3f}")
        print(f"Max took time (seconds): {max_took_time:.3f}")
        print(f"Avg took time (seconds): {avg_took_time:.3f}")

        print(f"Min service time (seconds): {min_service_time:.3f}")
        print(f"Max service time (seconds): {max_service_time:.3f}")
        print(f"Avg service time (seconds): {avg_service_time:.3f}")

        # Print the average response time
        print(f"Avg response time (seconds): {avg_response_time:.3f}")

        # Output the total number of queries ran
        print(f"Total number of queries ran: {total_queries_ran}")

        # Output the total runtime
        print(f"Total runtime (seconds): {total_runtime:.3f}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Run the async function
#await main()

# Ensure the client is properly closed after main completes
    await es.close()

if __name__ == "__main__":
    asyncio.run(main())


Total number of documents in the index 'tech15m2_random': 15610000
Number of primary shards: 3
Number of replicas: 1
Min took time (seconds): 0.004
Max took time (seconds): 0.241
Avg took time (seconds): 0.015
Min service time (seconds): 0.135
Max service time (seconds): 0.795
Avg service time (seconds): 0.157
Avg response time (seconds): 0.002
Total number of queries ran: 10000
Total runtime (seconds): 16.539
